# Exploratory Data Analysis (EDA)

Analisis de la red y preparación para la clasificación

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import random
import csv
import pickle

In [ ]:
edges = open('/content/drive/MyDrive/ES_Twitch_Dataset/ES_edges.csv')
next(edges)
grafo = nx.read_edgelist(edges, create_using=nx.DiGraph(), delimiter=',', nodetype=int)

Datos de la red:

In [ ]:
print("Nodos:", len(grafo.nodes))
print("Aristas:", len(grafo.edges))

Nodos: 4648
Aristas: 59382


In [ ]:
in_degree = list(dict(grafo.in_degree()).values())
total = sum(np.array(in_degree) == 0)
print("Cantidad de nodos sin seguidores: ", total)
print("Grado promedio de seguidores: ", sum(np.array(in_degree))  / len(in_degree))
print("Porcentaje de nodos sin seguidores: ", total * 100 / len(in_degree))

Cantidad de nodos sin seguidores:  599
Grado promedio de seguidores:  12.775817555938039
Porcentaje de nodos sin seguidores:  12.887263339070568


In [ ]:
out_degree = list(dict(grafo.out_degree()).values())
total = sum(np.array(out_degree) == 0)
print("Cantidad de nodos que no siguen a nadie: ", total)
print("Grado promedio de siguiendo: ", sum(np.array(out_degree))  / len(out_degree))
print("Porcentaje de nodos que no siguen a nadie: ", total * 100 / len(out_degree))

Cantidad de nodos que no siguen a nadie:  620
Grado promedio de siguiendo:  12.775817555938039
Porcentaje de nodos que no siguen a nadie:  13.33907056798623


In [ ]:
print("Nodo con más seguidores: ", max(in_degree))
print("Nodo que mas sigue: ", max(out_degree))

Nodo con más seguidores:  809
Nodo que mas sigue:  618


# Preparación de la red para la clasificación

En este momento, solo tenemos nodos que están conectados. Al ser un problema de clasificación, vamos a necesitar nodos que no esten conectados. Así que voy a generar algunos de modo random, los mismos van a tener por lo menos distancia 2 de camino más corto entre ellos.

In [ ]:
pd.read_csv('/content/drive/MyDrive/ES_Twitch_Dataset/ES_edges.csv').to_csv('/content/drive/MyDrive/ES_Twitch_Dataset/aux.csv', header = False, index = False)
df = csv.reader(open('/content/drive/MyDrive/ES_Twitch_Dataset/aux.csv', 'r'))
edges = dict() 
for edge in df:
    edges[(edge[0], edge[1])] = 1 #cargo las conexiones que ya existen

In [ ]:
missing_edges = set([])

# Como maximo voy a tener la misma cantidad de conexiones y no conexiones.
while (len(missing_edges) < len(grafo.edges)):
    a=random.randint(1, len(grafo.nodes))
    b=random.randint(1, len(grafo.nodes))
    tmp = edges.get((a, b), -1)
    if tmp == -1 and a != b:
        try:
            if nx.shortest_path_length(grafo, source = a, target = b) > 2: 
                missing_edges.add((a,b))
            else:
                continue  
        except:  
                missing_edges.add((a,b))              
    else:
        continue

In [ ]:
len(missing_edges)

59382

In [ ]:
missing_df = pd.DataFrame(list(missing_edges), columns = ['from', 'to']) 
missing_df.to_csv('/content/drive/MyDrive/ES_Twitch_Dataset/missing_edges.csv') #lo guardo por las dudas

Ahora junto ambos dataframes, para quedarme con el final.

In [ ]:
edges_df = pd.read_csv('/content/drive/MyDrive/ES_Twitch_Dataset/ES_edges.csv', encoding='latin-1')
edges_df['Target'] = 1
missing_df['Target'] = 0

In [ ]:
df_final = pd.concat([edges_df, missing_df])
df_final.rename(columns={'from': 'Source', 'to': 'Destination'}, inplace = True)
df_final.head()

,Source,Destination,Target
0,0,1819,1
1,0,2840,1
2,1,1565,1
3,1,1309,1
4,1,1397,1


In [ ]:
df_final.shape

(118764, 3)

In [ ]:
df_final.to_csv('/content/drive/MyDrive/ES_Twitch_Dataset/Final_Edges.csv')

Teniendo esto ya puedo avanzar con el Featuring